In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def write_to_file(data, filename):
    with open(filename, 'w') as f:
        for line in data:
            f.write(line + '\n')

def read_from_file(filename):
    with open(filename, 'r') as f:
        return np.array(f.read().splitlines())
    

In [3]:
train = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\train.csv").fillna('')
test = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test.csv").fillna('')
submission = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\sample_submission.csv")
print("Records in training set:",len(train))
print("Records in test set:",len(test))
train

Records in training set: 159571
Records in test set: 153164


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [4]:
y_train = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test_labels.csv")
classes = y_train.columns.values[1:]
classes 


array(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'], dtype=object)

%cd D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data

In [5]:
X_train = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_train_cleaned_vanilla.txt")
X_test = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_test_cleaned_vanilla.txt")
X_train.shape
X_train

array(['explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now    ',
       'daww he matches this background colour i am seemingly stuck with thanks talk   january   utc',
       'hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info',
       ...,
       'spitzer umm theres no actual article for prostitution ring crunch captain',
       'and it looks like it was actually you who put on the speedy to have the first version deleted now that i look at it',
       'and i really do not think you understand i came here and my idea was bad right away what kind of community goes you have bad ideas go away instead of helping r

In [6]:
train.loc[:, 'comment_text'] = X_train
train

test.loc[:, 'comment_text'] = X_test
test

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland
3,00017563c3f7919a,if you have a look back at the source the info...
4,00017695ad8997eb,i do not anonymously edit articles at all
...,...,...
153159,fffcd0960ee309b5,i totally agree this stuff is nothing but too ...
153160,fffd7a9a6eb32c16,throw from out field to home plate does it get...
153161,fffda9e8d6fafa9e,okinotorishima categories i see your changes a...
153162,fffe8f1340a79fc2,one of the founding nations of the eu germany ...


In [7]:
from sklearn.model_selection import train_test_split
# from nltk import ngrams, bigrams, trigrams
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

def cv_tf_train_test(dataframe, label, vectorizer, ngram):
    # Split the data to X and y data sets
    X = dataframe.comment_text
    y = dataframe[label]

    # Split data to training and test data
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=50)
    cv1 = vectorizer(ngram_range = (ngram))

    X_train_cv1 = cv1.fit_transform(X_train)
    X_test_cv1 = cv1.transform(X_test)

    svm_model = LinearSVC()
    svm_model.fit(X_train_cv1, y_train)
    f1_score_data = {'F1 Score':[f1_score(svm_model.predict(X_test_cv1), y_test)]}
    df_f1 = pd.DataFrame(f1_score_data, index = ['SVM'])
    return df_f1

In [8]:
Insulting_comment_train = train.loc[:, ['id', 'comment_text', 'insult']]

IdentityHate_comment_train = train.loc[:, ['id', 'comment_text', 'identity_hate']]

Obscene_comment_train = train.loc[:, ['id', 'comment_text', 'obscene']]

Threatening_comment_train = train.loc[:, ['id', 'comment_text', 'threat']]

Severetoxic_comment_train = train.loc[:, ['id', 'comment_text', 'severe_toxic']]

Toxic_comment_train = train.loc[:, ['id', 'comment_text', 'toxic']]

In [9]:
Toxic_comment_train['toxic'].value_counts()


toxic
0    144277
1     15294
Name: count, dtype: int64

In [10]:
Toxic_comment_balanced_0 = Toxic_comment_train[Toxic_comment_train['toxic'] == 0].iloc[0:5000, :]
Toxic_comment_balanced_1 = Toxic_comment_train[Toxic_comment_train['toxic'] == 1].iloc[0:5000, :]
Toxic_comment_balanced = pd.concat([Toxic_comment_balanced_1,Toxic_comment_balanced_0])
Toxic_comment_balanced['toxic'].value_counts()


toxic
1    5000
0    5000
Name: count, dtype: int64

In [11]:
Severetoxic_comment_train['severe_toxic'].value_counts()

severe_toxic
0    157976
1      1595
Name: count, dtype: int64

In [12]:
Severetoxic_comment_balanced_0 = Severetoxic_comment_train[Severetoxic_comment_train['severe_toxic'] == 0].iloc[0:1595, :]
Severetoxic_comment_balanced_1 = Severetoxic_comment_train[Severetoxic_comment_train['severe_toxic'] == 1].iloc[0:1595, :]
Severetoxic_comment_balanced = pd.concat([Severetoxic_comment_balanced_1,Severetoxic_comment_balanced_0])
Severetoxic_comment_balanced['severe_toxic'].value_counts()


severe_toxic
1    1595
0    1595
Name: count, dtype: int64

In [13]:
Obscene_comment_train['obscene'].value_counts()

obscene
0    151122
1      8449
Name: count, dtype: int64

In [14]:
Obscene_comment_balanced_0 = Obscene_comment_train[Obscene_comment_train['obscene'] == 0].iloc[0:5000, :]
Obscene_comment_balanced_1 = Obscene_comment_train[Obscene_comment_train['obscene'] == 1].iloc[0:5000, :]
Obscene_comment_balanced = pd.concat([Obscene_comment_balanced_1,Obscene_comment_balanced_0])
Obscene_comment_balanced['obscene'].value_counts()


obscene
1    5000
0    5000
Name: count, dtype: int64

In [15]:
Threatening_comment_train['threat'].value_counts()

threat
0    159093
1       478
Name: count, dtype: int64

In [16]:
Threatening_comment_balanced_0 = Threatening_comment_train[Threatening_comment_train['threat'] == 0].iloc[0:478, :]
Threatening_comment_balanced_1 = Threatening_comment_train[Threatening_comment_train['threat'] == 1].iloc[0:478, :]
Threatening_comment_balanced = pd.concat([Threatening_comment_balanced_1,Threatening_comment_balanced_0])
Threatening_comment_balanced['threat'].value_counts()


threat
1    478
0    478
Name: count, dtype: int64

In [17]:
Insulting_comment_train['insult'].value_counts()

insult
0    151694
1      7877
Name: count, dtype: int64

In [18]:
Insulting_comment_balanced_0 = Insulting_comment_train[Insulting_comment_train['insult'] == 0].iloc[0:5000, :]
Insulting_comment_balanced_1 = Insulting_comment_train[Insulting_comment_train['insult'] == 1].iloc[0:5000, :]
Insulting_comment_balanced = pd.concat([Insulting_comment_balanced_1,Insulting_comment_balanced_0])
Insulting_comment_balanced['insult'].value_counts()


insult
1    5000
0    5000
Name: count, dtype: int64

In [19]:
IdentityHate_comment_train['identity_hate'].value_counts()

identity_hate
0    158166
1      1405
Name: count, dtype: int64

In [20]:
IdentityHate_comment_balanced_0 = IdentityHate_comment_train[IdentityHate_comment_train['identity_hate'] == 0].iloc[0:1405, :]
IdentityHate_comment_balanced_1 = IdentityHate_comment_train[IdentityHate_comment_train['identity_hate'] == 1].iloc[0:1405, :]
IdentityHate_comment_balanced = pd.concat([IdentityHate_comment_balanced_1,IdentityHate_comment_balanced_0])
IdentityHate_comment_balanced['identity_hate'].value_counts()


identity_hate
1    1405
0    1405
Name: count, dtype: int64

In [21]:
Toxic_comment_cv = cv_tf_train_test(Toxic_comment_balanced,'toxic', TfidfVectorizer, (1,1))
Toxic_comment_cv.rename(columns={'F1 Score': 'F1 Score(Toxic)'}, inplace = True)
Toxic_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(Toxic)
SVM,0.891919


In [22]:
severe_toxic_comment_cv = cv_tf_train_test(Severetoxic_comment_balanced,'severe_toxic', TfidfVectorizer, (1,1))
severe_toxic_comment_cv.rename(columns={'F1 Score': 'F1 Score(severe_toxic)'}, inplace = True)
severe_toxic_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(severe_toxic)
SVM,0.927632


In [23]:
obscene_comment_cv = cv_tf_train_test(Obscene_comment_balanced,'obscene', TfidfVectorizer, (1,1))
obscene_comment_cv.rename(columns={'F1 Score': 'F1 Score(obscene)'}, inplace = True)
obscene_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(obscene)
SVM,0.910186


In [24]:
threat_comment_cv = cv_tf_train_test(Threatening_comment_balanced,'threat', TfidfVectorizer, (1,1))
threat_comment_cv.rename(columns={'F1 Score': 'F1 Score(threat)'}, inplace = True)
threat_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(threat)
SVM,0.945652


In [25]:
insult_comment_cv = cv_tf_train_test(Insulting_comment_balanced,'insult', TfidfVectorizer, (1,1))
insult_comment_cv.rename(columns={'F1 Score': 'F1 Score(insult)'}, inplace = True)
insult_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(insult)
SVM,0.907606


In [26]:
identity_hate_comment_cv = cv_tf_train_test(IdentityHate_comment_balanced,'identity_hate', TfidfVectorizer, (1,1))
identity_hate_comment_cv.rename(columns={'F1 Score': 'F1 Score(identity_hate)'}, inplace = True)
identity_hate_comment_cv

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


,F1 Score(identity_hate)
SVM,0.910092


In [27]:
X = train['comment_text']
print(X)
y = pd.read_csv('D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\labels.csv')
print(y)

0         explanation why the edits made under my userna...
1         daww he matches this background colour i am se...
2         hey man i am really not trying to edit war it ...
3         more i cannot make any real suggestions on imp...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself that is a ho...
159568    spitzer umm theres no actual article for prost...
159569    and it looks like it was actually you who put ...
159570    and i really do not think you understand i cam...
Name: comment_text, Length: 159571, dtype: object
        toxic  severe_toxic  obscene  threat  insult  identity_hate
0           0             0        0       0       0              0
1           0             0        0       0       0              0
2           0             0        0       0       0              0
3           0     

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True, smooth_idf=True, sublinear_tf=True)
train_doc = vec.fit_transform(X_train)
test_doc = vec.transform(X_test)
vec.get_feature_names_out()

array(['aa', 'aa talk', 'aa to', ..., 'zzz', 'zzzz', 'zzzzz'],
      dtype=object)

'\nhttps://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/\nSupport Vector Machines were designed for binary classification and \ndo not natively support classification tasks with more than two classes.\n\nThe SVM is really rigid, a.k.a. relatively high bias, in terms of the function that is learned: \none line separating two classes from each other. There is simply no way that multiple classes can be learned. \nThis is why, next to multiclass classification, multilabel classification cannot be performed natively with SVMs.\n'

In [30]:
# X_train, X_valid, y_train, y_valid = train_test_split(Toxic_comment_balanced.comment_text, Toxic_comment_balanced['toxic'], test_size=0.2, random_state= 42)
# tfv = TfidfVectorizer(ngram_range= (1,1))
# X_train_fit = tfv.fit_transform(X_train)
# X_test_fit = tfv.transform(X_test)
# svc_model = LinearSVC()
# svc_model.fit(X_train_fit, y_train)
# svc_model.score(svc_model.fit(X_train_fit, y_train))
# print(svc_model.predict(X_test_fit).shape)

In [31]:
X_train, X_valid = train_test_split(train, train_size=0.8, random_state=42)
tfv = TfidfVectorizer(ngram_range= (1,1))
X_train_fit = tfv.fit_transform(X_train['comment_text'])
X_valid_fit = tfv.transform(X_valid['comment_text'])
X_test_fit = tfv.transform(test['comment_text'])

In [32]:
from sklearn.calibration import CalibratedClassifierCV
def pr(y_i, y):
    p = X_train_fit[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    model = CalibratedClassifierCV(LinearSVC(penalty='l2', dual=False), cv=3)
    x_nb = X_train_fit.multiply(r)
    return model.fit(x_nb, y), r

In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, train_size=0.8, random_state=42)

In [35]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_labels = X_train.drop(['id', 'comment_text'], axis = 1)
valid_labels = X_valid.drop(['id', 'comment_text'], axis = 1)


In [36]:
model = []
for i, col in enumerate(classes):
    print(col)
    model.append(get_mdl(train_labels[col]))
    preds= model[i][0].predict_proba(X_valid_fit.multiply(model[i][1]))[:,1].reshape(-1, 1)
    print(roc_auc_score(valid_labels, preds))

toxic
0.9742125245982952
severe_toxic
0.8766394082240262
obscene
0.9307571655489074
threat
0.7434680096983546
insult
0.9475288056535578
identity_hate
0.8654542479874617


In [37]:
preds = np.zeros((len(test), len(classes)))

for i, col in enumerate(classes):
    print(col)
    preds[:, i] = model[i][0].predict_proba(X_test_fit.multiply(model[i][1]))[:, 1]

toxic
severe_toxic
obscene
threat
insult
identity_hate


In [38]:
submid = pd.DataFrame({'id': submission["id"]})
SUB = pd.concat([submid, pd.DataFrame(preds, columns = classes)], axis=1)
SUB.to_csv('submission.csv', index=False)
